In [1]:
import importlib
import matplotlib.pyplot as plt
import os
import pickle
import photogrammetry as pg

# plt.style.use('seaborn-poster')
plt.style.use('default')

%matplotlib qt

In [2]:
importlib.reload(pg)

<module 'photogrammetry' from '/home/evanmayer/github/hotspot/verification/photogrammetry.py'>

In [3]:
# check for pickled camera matrices to avoid expensive recalibration
if not (
    os.path.exists('camera_cal_mtx.pickle') and
    os.path.exists('camera_cal_dist.pickle') and
    os.path.exists('camera_cal_optimal_camera_matrix.pickle')
):
    # calibrate the camera for distortion
    mtx, dist, optimal_camera_matrix, roi = pg.calibrate_camera(
        os.path.join('input', 'camera_cal', 'charuco'),
        method='charuco',
        plot=False
    )
    with open('camera_cal_mtx.pickle', 'wb') as f:
        pickle.dump(mtx, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('camera_cal_dist.pickle', 'wb') as f:
        pickle.dump(dist, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open('camera_cal_optimal_camera_matrix.pickle', 'wb') as f:
        pickle.dump(optimal_camera_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('camera_cal_mtx.pickle', 'rb') as f:
        mtx = pickle.load(f)
    with open('camera_cal_dist.pickle', 'rb') as f:
        dist = pickle.load(f)
    with open('camera_cal_optimal_camera_matrix.pickle', 'rb') as f:
        optimal_camera_matrix = pickle.load(f)

In [4]:
image_data_charuco_test = pg.find_targets(
    os.path.join('input', 'meas', 'charuco_test'),
    os.path.join('input', 'targets'),
    mtx,
    dist,
    optimal_camera_matrix,
    stride=1,
    plot=False
)

INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501.JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501 (copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501 (another copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9502 (another copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9502 (3rd copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501 (5th copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501 (4th copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9502 (6th copy).JPG: True
INFO:photogrammetry:Corner detection after de-distortion in input/meas/charuco_test/IMG_9501 (3

In [5]:
# print(image_data)

In [11]:
plt.close('all')

In [12]:
importlib.reload(pg)

<module 'photogrammetry' from '/home/evanmayer/github/hotspot/verification/photogrammetry.py'>

In [14]:
command_file = os.path.join('..', 'data', 'input', 'profiles', '24in_breadboard_raster_7x7_.15x.15.csv')
pg.post_process_many_to_many(image_data_charuco_test, command_file, ref='SW')
# pg.post_process_many_to_one(image_data_charuco_test)

INFO:photogrammetry:dist of raft_target from SW
	mean 0.2188231145998646
	std 0.00017337256997134167
	std (mm) 0.17337256997134168
	% err 0.07922955044688361 %
INFO:photogrammetry:8 + 19.68/32"
INFO:photogrammetry:dist of SE_target from SW
	mean 0.4605444654392554
	std 0.00018593309948051204
	std (mm) 0.18593309948051204
	% err 0.04037245335326608 %
INFO:photogrammetry:18 + 4.21/32"
INFO:photogrammetry:dist of NW_target from SW
	mean 0.5195596119401296
	std 0.00037403624736442317
	std (mm) 0.37403624736442315
	% err 0.07199101677047302 %
INFO:photogrammetry:20 + 14.56/32"
INFO:photogrammetry:dist of NE_target from SW
	mean 0.6629498300922072
	std 0.00018706161573560384
	std (mm) 0.18706161573560384
	% err 0.028216556856132856 %
INFO:photogrammetry:26 + 3.21/32"
